In [1]:
using Pkg, Revise

In [2]:
pwd()

"/Users/hunt/GitHub/AgoUtils.jl"

In [1]:
Pkg.activate("../")
using AgoUtils

  Activating project at `~/GitHub/AgoUtils.jl`


---

---

## Creating Samplers

### General sampler creation

This is generally how to build a weighted sampler:

In [4]:
using BioSequences

const sampler_order = RNA[RNA_G, RNA_C, RNA_A, RNA_U]
SamplerWeighted(sampler_order, [0.125, 0.125, 0.75/2])

SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.125, 0.125, 0.375, 0.375])

But the goal is to vectorize this and consolidate the process.

---

### Mapped Sampler Creation

In [3]:
nms = ([Symbol("gc50_", string(i)) for i = 1:2]...,)
# assuming G, C, A, and U for order
vals = (repeat([fill(0.5/2, 3)], 2))
# probabilities = NamedTuple{nms}(vals)
probabilities = merge(
    (gc40_1 = [0.2, 0.2, 0.3],), # trailing comma defines the tuple type
    (; zip(nms, vals)...), # programmatic zip splat kwarg style construction
    (; :gc60_1 => [0.3, 0.3, 0.2]) # kwarg style named tuple key/value pair
)

(gc40_1 = [0.2, 0.2, 0.3], gc50_1 = [0.25, 0.25, 0.25], gc50_2 = [0.25, 0.25, 0.25], gc60_1 = [0.3, 0.3, 0.2])

In [4]:
([Symbol("gc50_", string(i)) for i = 1:4]...,) |> typeof

    ╭─────────────────────────────────────────────────────────────────────────╮
    │                                                                         │
    │                              NTuple <: Any                              │
    │                                │1 ::Symbol                              │
    │                                │2 ::Symbol                              │
    │                                │3 ::Symbol                              │
    │                                │4 ::Symbol                              │
    │                                                                         │
    ╰─────────────────────────────────────────────────────────────────────────╯
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Docstring ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    ┌──────────────────────────────────────────────────────────────────┐        
    │  Tuple{{Types...}}                                               │        
    └────────────────────────────────

The same thing can be achieved with an abbreviated syntax as shown below.

In [5]:
(; zip(nms, vals)...)

(gc50_1 = [0.25, 0.25, 0.25], gc50_2 = [0.25, 0.25, 0.25])

In [6]:
(; zip(
    ([Symbol("gc50_", string(i)) for i = 1:4]), 
    repeat([fill(0.5/2, 3)], 4)
)...)

(gc50_1 = [0.25, 0.25, 0.25], gc50_2 = [0.25, 0.25, 0.25], gc50_3 = [0.25, 0.25, 0.25], gc50_4 = [0.25, 0.25, 0.25])

In [7]:
probabilities

(gc40_1 = [0.2, 0.2, 0.3], gc50_1 = [0.25, 0.25, 0.25], gc50_2 = [0.25, 0.25, 0.25], gc60_1 = [0.3, 0.3, 0.2])

In [9]:
samplers = map(((a,b,c),) -> SamplerWeighted(sampler_order, [a,b,c]), probabilities)

(gc40_1 = SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.2, 0.2, 0.3, 0.30000000000000004]), gc50_1 = SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.25, 0.25, 0.25, 0.25]), gc50_2 = SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.25, 0.25, 0.25, 0.25]), gc60_1 = SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.3, 0.3, 0.2, 0.19999999999999996]))

This can also be done using a `do` syntax block as shown below.

In [10]:
map(probabilities) do (a,b,c)
    SamplerWeighted(sampler_order, [a,b,c])
end

(gc40_1 = SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.2, 0.2, 0.3, 0.30000000000000004]), gc50_1 = SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.25, 0.25, 0.25, 0.25]), gc50_2 = SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.25, 0.25, 0.25, 0.25]), gc60_1 = SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.3, 0.3, 0.2, 0.19999999999999996]))

Or with formal function syntax..

In [11]:
#=
map(function f((a,b,c)).. -- named
	or
map(function ((a,b,c),).. -- pseudo-anonymous?
	or..
=#
map(function f((a,b,c),) # -- combo of above syntax
		SamplerWeighted(rna"GCAU", [a,b,c])
end, probabilities)

(gc40_1 = SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.2, 0.2, 0.3, 0.30000000000000004]), gc50_1 = SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.25, 0.25, 0.25, 0.25]), gc50_2 = SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.25, 0.25, 0.25, 0.25]), gc60_1 = SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.3, 0.3, 0.2, 0.19999999999999996]))

**Note for above:** *You either have to put a comma after the tuple for argument expansion (like in the anonymous stabby function) beacuse this is how Julia defines the tuple (not with parenthesis), or name the function because.. I'm not sure why that works, but it does. Maybe just do both as shown above to be safe.* 😉

---

---

## Generating Random (RNA) Sequences

First we can set the length of sequences we would like to create. The maximum length IDT will provide for a custom RNA oligo is 60nt/bp for a *standard* synthesis and 120nt/bp for an *ultramer* synthesis. I think going shorter will be preferred for managing plates of guides. A 38mer will allow 24 x 16nt guides (if the guide which overhangs the 3ʹ end is included in this set: (38-16)+1+1=24), which works out well to screen variation in the 5ʹ-phosphorylated base. All combinations would yield 96 guides per substrate.

In [12]:
const oligoLength = 38

38

In [13]:
using Random

In [14]:
myseeds = [1234, 2345, 4567, 7890]
myrngs = Random.MersenneTwister.(myseeds)

╭─────────────────────────────────────╮
│                                     │
│     (1)   MersenneTwister(1234)     │
│     (2)   MersenneTwister(2345)     │
│     (3)   MersenneTwister(4567)     │
│     (4)   MersenneTwister(7890)     │
│                                     │
│                                     │
╰───────────────────────── 4 items ───╯


In [15]:
myrngs[1]

MersenneTwister(1234)

In [16]:
for i in 1:4
    println(rand(myrngs[i], samplers[1], 10))
end

RNA[RNA_A, RNA_U, RNA_A, RNA_A, RNA_U, RNA_U, RNA_C, RNA_C, RNA_C, RNA_A]
RNA[RNA_C, RNA_C, RNA_U, RNA_C, RNA_U, RNA_A, RNA_A, RNA_A, RNA_C, RNA_G]
RNA[RNA_A, RNA_G, RNA_G, RNA_U, RNA_G, RNA_A, RNA_G, RNA_U, RNA_C, RNA_U]
RNA[RNA_A, RNA_G, RNA_U, RNA_U, RNA_A, RNA_A, RNA_A, RNA_C, RNA_U, RNA_C]


In [17]:
myrng = Random.MersenneTwister()

MersenneTwister(0xc03e17f97973c86d8b07144e2209596c)

In [18]:

substrates_MT = map(samplers, myrngs) do smplr, rng
    println(rng)
    println(smplr)
    randseq(rng, RNAAlphabet{4}(), smplr, oligoLength)
end
# map(sampler -> randseq(Random.MersenneTwister(1234), RNAAlphabet{4}(), sampler, oligoLength), samplers)

MersenneTwister(1234, (0, 1002, 0, 10))
SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.2, 0.2, 0.3, 0.30000000000000004])
MersenneTwister(2345, (0, 1002, 0, 10))
SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.25, 0.25, 0.25, 0.25])
MersenneTwister(4567, (0, 1002, 0, 10))
SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.25, 0.25, 0.25, 0.25])
MersenneTwister(7890, (0, 1002, 0, 10))
SamplerWeighted{RNA}(RNA[RNA_G, RNA_C, RNA_A, RNA_U], [0.3, 0.3, 0.2, 0.19999999999999996])


╭──────────────────────────────────────────────────────╮
│                                                      │
│     (1)   AGGUAGCAGUUUUUUGGCGCUACGGAUCAUGACCCGGC     │
│     (2)   GUAGGGUCCAUGAUCUUAUUGCAUUCUUGUCAUUGGCU     │
│     (3)   UAAUACAGAUGUUGCUGAUCAAUGUAGCUGAUGUGCGU     │
│     (4)   CCCGUCUGGCGCUGUCCCGGGCUUGCUCAGUUGCACGA     │
│                                                      │
│                                                      │
╰────────────────────────────────────────── 4 items ───╯


In [31]:
substrates = map(sampler -> randseq(RNAAlphabet{4}(), sampler, oligoLength), samplers)

(gc40_1 = GAACUCCGCUAGUUCACAGGUAAUACAAGGAUACAUGA, gc50_1 = CAUAGUCGAGCUUGAUUCUCCCCCCGGUACCAAAUGAA, gc50_2 = CCGUACUCCUUUACUGUUUCCCUAAGUCUCUCCCAACA, gc60_1 = GCGUCCGGGAUUGCCUCGGAGAGCGCUGGCACCGCAGU)

---

---

## Generating Guides for the Substrates

In [32]:
using AgoUtils

In [33]:
const guideLength = 16

16

In [34]:
guides = map(substrates) do seq
    makeguides(seq, guideLength, DNAAlphabet{2})
end

(gc40_1 = NucleicAcidGuide[GuideDNA{2}(DNAAlphabet{2}, TGAACTAGCGGAGTTC, 16, Float16(0.5), DNA_T, NucleicAcid[DNA_A, DNA_C, DNA_G], LongSequence{DNAAlphabet{2}}[AGAACTAGCGGAGTTC, CGAACTAGCGGAGTTC, GGAACTAGCGGAGTTC]), GuideDNA{2}(DNAAlphabet{2}, GTGAACTAGCGGAGTT, 16, Float16(0.5), DNA_G, NucleicAcid[DNA_A, DNA_C, DNA_T], LongSequence{DNAAlphabet{2}}[ATGAACTAGCGGAGTT, CTGAACTAGCGGAGTT, TTGAACTAGCGGAGTT]), GuideDNA{2}(DNAAlphabet{2}, TGTGAACTAGCGGAGT, 16, Float16(0.5), DNA_T, NucleicAcid[DNA_A, DNA_C, DNA_G], LongSequence{DNAAlphabet{2}}[AGTGAACTAGCGGAGT, CGTGAACTAGCGGAGT, GGTGAACTAGCGGAGT]), GuideDNA{2}(DNAAlphabet{2}, CTGTGAACTAGCGGAG, 16, Float16(0.5625), DNA_C, NucleicAcid[DNA_A, DNA_G, DNA_T], LongSequence{DNAAlphabet{2}}[ATGTGAACTAGCGGAG, GTGTGAACTAGCGGAG, TTGTGAACTAGCGGAG]), GuideDNA{2}(DNAAlphabet{2}, CCTGTGAACTAGCGGA, 16, Float16(0.5625), DNA_C, NucleicAcid[DNA_A, DNA_G, DNA_T], LongSequence{DNAAlphabet{2}}[ACTGTGAACTAGCGGA, GCTGTGAACTAGCGGA, TCTGTGAACTAGCGGA]), GuideDNA{2}(DNAAl

In [35]:
substrates.gc50_1

38nt RNA Sequence:
CAUAGUCGAGCUUGAUUCUCCCCCCGGUACCAAAUGAA

In [42]:
guides.gc50_1[1]

16nt DNA Sequence:
ATCAAGCTCGACTATG
43.75% GC


In [37]:
using DataFrames

In [38]:
AgoUtils._wells_96

╭───────────────────╮
│                   │
│      (1)   A1     │
│      (2)   B1     │
│      (3)   C1     │
│      (4)   D1     │
│      (5)   E1     │
│      (6)   F1     │
│      (7)   G1     │
│      (8)   H1     │
│      (9)   A2     │
│     (10)   B2     │
│     (11)   C2     │
│     (12)   D2     │
│     (13)   E2     │
│     (14)   F2     │
│     (15)   G2     │
│     (16)   H2     │
│     (17)   A3     │
│     (18)   B3     │
│     (19)   C3     │
│     (20)   D3     │
│     (21)   E3     │
│     (22)   F3     │
│     (23)   G3     │
│     (24)   H3     │
│     (25)   A4     │
│     (26)   B4     │
│     (27)   C4     │
│     (28)   D4     │
│     (29)   E4     │
│     (30)   F4     │
│     (31)   G4     │
│     (32)   H4     │
│     (33)   A5     │
│     (34)   B5     │
│     (35)   C5     │
│     (36)   D5     │
│     (37)   E5     │
│     (38)   F5     │
│     (39)   G5     │
│     (40)   H5     │
│     (41)   A6     │
│     (42)   B6     │
│     (43)   C6     │
│     (44)

In [39]:
wellseqs = reshape(mapreduce(AgoUtils._fetchseqs, vcat, vcat(guides...)), (24,16))

╭──── Matrix {LongSequence  ────────────────────────────────╮
│                                                                             │
│                                                                             │
│              (1)            (2)            (3)            (4)        (16)   │
│                                                                             │
│   (1)    TGAACTAGC...   TACCTGTGA...   TTGTATTAC...   GTATCCTTG...   ⋯      │
│   (2)    AGAACTAGC...   AACCTGTGA...   ATGTATTAC...   ATATCCTTG...   ⋯      │
│   (3)    CGAACTAGC...   CACCTGTGA...   CTGTATTAC...   CTATCCTTG...   ⋯      │
│   (4)    GGAACTAGC...   GACCTGTGA...   GTGTATTAC...   TTATCCTTG...   ⋯      │
│   (5)    GTGAACTAG...   TTACCTGTG...   CTTGTATTA...   TGTATCCTT...   ⋯      │
│   (6)    ATGAACTAG...   ATACCTGTG...   ATTGTATTA...   AGTATCCTT...   ⋯      │
│   (7)    CTGAACTAG...   CTACCTGTG...   GTTGTATTA...   CGTATCCTT...   ⋯      │
│   (8)    TTGAACTAG...   GTACCTGTG...   TTTGTATTA...   GG

In [40]:
wellids = reshape(AgoUtils._wells_384, (24,16))

╭──── Matrix   ───────────────────╮
│                                         │
│                                         │
│          (1)   (2)   (3)   (4)   (16)   │
│                                         │
│   (1)    A1    A4    A7    A10   ⋯      │
│   (2)    B1    B4    B7    B10   ⋯      │
│   (3)    C1    C4    C7    C10   ⋯      │
│   (4)    D1    D4    D7    D10   ⋯      │
│   (5)    E1    E4    E7    E10   ⋯      │
│   (6)    F1    F4    F7    F10   ⋯      │
│   (7)    G1    G4    G7    G10   ⋯      │
│   (8)    H1    H4    H7    H10   ⋯      │
│   (9)    A2    A5    A8    A11   ⋯      │
│   (10)   B2    B5    B8    B11   ⋯      │
│   (11)   C2    C5    C8    C11   ⋯      │
│   (12)   D2    D5    D8    D11   ⋯      │
│   (13)   E2    E5    E8    E11   ⋯      │
│   (14)   F2    F5    F8    F11   ⋯      │
│   (15)   G2    G5    G8    G11   ⋯      │
│   (16)   H2    H5    H8    H11   ⋯      │
│   (17)   A3    A6    A9    A12   ⋯      │
│   (18)   B3    B6    B9    B12   ⋯    

In [41]:
Dict(wellids[i] => wellseqs[i] for i=1:384)

╭──── Dict {String, LongSequence  ──────╮
│                                                         │
│      I5  => GGGGGAGAATCAAGCT      │
│      I7  => ACCGGGGGGAGAATCA      │
│      D20 => TAGACTTAGGGAAACA      │
│      O4  => CGGGAGAATCAAGCTC      │
│      F11 => AATGTATCCTTGTATT      │
│      A5  => ATTACCTGTGAACTAG      │
│      I22 => CGGTGCCAGCGCTCTC      │
│      A18 => CTTAGGGAAACAGTAA      │
│      M5  => GGGGGGAGAATCAAGC      │
│      D12 => GCATGTATCCTTGTAT      │
│      F20 => CGAGACTTAGGGAAAC      │
│      D2  => GGTGAACTAGCGGAGT      │
│      C24 => CGTTGGGAGAGACTTA      │
│      O20 => CGCCAGCGCTCTCCGA      │
│      K24 => GCTGCGGTGCCAGCGC      │
│      E10 => TGTATCCTTGTATTAC      │
│      D21 => TAGAGACTTAGGGAAA      │
│      F4  => ATACCTGTGAACTAGC      │
│      O7  => CACCGGGGGGAGAATC      │
│      J19 => AAGCGCTCTCCGAGGC      │
│      F10 => AGTATCCTTGTATTAC      │
│      J3  => AAGAATCAAGCTCGAC      │
│      J14 => AGAGGCAATCCCGGAC      │
│      B15 => AAAACAGTAA